In [27]:
# CONDA: tnt
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import helpers as h
from datetime import timedelta
reload(h)

<module 'helpers' from '/Users/koenraijer/Documents/Universiteit/Student-assistent/IP-TNT/Data/helpers.py'>

# Obtaining Inquisit timestamps

In [14]:
dir_list = h.get_dir_list('input/calibration_pp20/inquisit')

In [43]:
reload(h)
inquisit_timestamps = []

for file in dir_list:
    inquisit_timestamps.extend(h.get_inquisit_tags(f"input/calibration_pp20/inquisit/{file}"))

inquisit_timestamps.sort()

print(inquisit_timestamps)
print(len(inquisit_timestamps))

# Convert timestamp in format 1681986695.444 to datetime: Timestamp('2023-04-20 10:31:35.444000')
pd.to_datetime(inquisit_timestamps[0], unit='s')

[1681986695.444, 1681986890.308, 1681988379.043, 1681989368.37, 1681990006.293, 1681991320.92, 1681991634.8700001, 1681992273.859, 1681992478.773, 1681993163.464, 1681994714.336, 1681994755.499, 1681994826.996, 1681994891.0700002, 1681994949.3409998, 1681995050.463, 1681995329.934, 1681995571.2649999]
18


Timestamp('2023-04-20 10:31:35.444000')

# Obtaining Empatica timestamps

In [19]:
dir_list2 = h.get_dir_list('input/calibration_pp20/empatica' )

In [45]:
# Read tags.csv
tags_df = pd.read_csv(f'input/calibration_pp20/empatica/{dir_list2[1]}/tags.csv', header=None, names=['time'])

empatica_timestamps = list(tags_df['time'])

print(empatica_timestamps)
print(len(empatica_timestamps))

# Convert timestamp in format 1681986694.02 to datetime: Timestamp('2023-04-20 10:31:34.020000')
pd.to_datetime(empatica_timestamps[0], unit='s')

[1681986694.02, 1681986888.64, 1681988378.56, 1681989366.97, 1681990004.83, 1681991319.67, 1681991633.8, 1681992272.84, 1681992477.62, 1681993162.73, 1681994713.58, 1681994754.84, 1681994825.25, 1681994890.52, 1681994948.69, 1681995049.58, 1681995328.86, 1681995570.55]
18


Timestamp('2023-04-20 10:31:34.020000')

# Calculating the difference between Inquisit and Empatica timestamps

In [55]:
reload(h)

# Calculate differences between Empatica timestamps and their closest Inquisit timestamps
differences = []
for empatica_ts in empatica_timestamps:
    closest_inquisit_ts = h.closest_timestamp(empatica_ts, inquisit_timestamps)
    diff = abs(empatica_ts - closest_inquisit_ts)
    
    # If the difference is within the 5-second threshold, add it to the list
    if diff <= 5:
        differences.append((empatica_ts, closest_inquisit_ts, diff))

# Calculate the average difference
average_difference = sum(diff for _, _, diff in differences) / len(differences)
print(average_difference)
print(len(differences))
print("Range:", min(differences, key=lambda x: x[2]), max(differences, key=lambda x: x[2]))
print(differences)

1.038777814971076
18
Range: (1681988378.56, 1681988379.043, 0.4830000400543213) (1681994825.25, 1681994826.996, 1.746000051498413)
[(1681986694.02, 1681986695.444, 1.4240000247955322), (1681986888.64, 1681986890.308, 1.6679999828338623), (1681988378.56, 1681988379.043, 0.4830000400543213), (1681989366.97, 1681989368.37, 1.3999998569488525), (1681990004.83, 1681990006.293, 1.4630000591278076), (1681991319.67, 1681991320.92, 1.25), (1681991633.8, 1681991634.8700001, 1.070000171661377), (1681992272.84, 1681992273.859, 1.0190000534057617), (1681992477.62, 1681992478.773, 1.1530001163482666), (1681993162.73, 1681993163.464, 0.7339999675750732), (1681994713.58, 1681994714.336, 0.7560000419616699), (1681994754.84, 1681994755.499, 0.6590001583099365), (1681994825.25, 1681994826.996, 1.746000051498413), (1681994890.52, 1681994891.0700002, 0.5500001907348633), (1681994948.69, 1681994949.3409998, 0.6509997844696045), (1681995049.58, 1681995050.463, 0.8830001354217529), (1681995328.86, 1681995329.

You are an expert Python programmer and data scientist, good at solving tough problems relating to the calibration of timestamps coming from different sources. If you think anything I suggested is not the best way to do it, please feel free to suggest a better way. Think step by step, and explain your reasoning. Provide code and comments.

I have two lists of timestamps in UNIX/epoch format (in seconds) coming from two different data sources that need to be calibrated. `empatica_timestamps` is coming from Empatica wristband tags, which are registered whenever users press the button on the wearable. They were instructed to do so at specific moments in time using blocks in Inquisit, software to do psychological experiments. The timing of those blocks are stored in `inquisit_timestamps`.

What I need:
- Create a list with differences between each Empatica timestamp and the closest Inquisit timestamp.
- If the difference is more than 5 seconds, discard it.
- Average all items in the differences list. 

empatica_timestamps:
```py
[1681986694.02, 1681986888.64, 1681988378.56, 1681989366.97, 1681990004.83, 1681991319.67, 1681991633.8, 1681992272.84, 1681992477.62, 1681993162.73, 1681994713.58, 1681994754.84, 1681994825.25, 1681994890.52, 1681994948.69, 1681995049.58, 1681995328.86, 1681995570.55]
```

inquisit_timestamps:
```py
[1681986695.444, 1681986890.308, 1681988379.043, 1681989368.37, 1681990006.293, 1681991320.92, 1681991634.8700001, 1681992273.859, 1681992478.773, 1681993163.464, 1681994714.336, 1681994755.499, 1681994826.996, 1681994891.0700002, 1681994949.3409998, 1681995050.463, 1681995329.934, 1681995571.2649999]
```

Note: do not copy the timestamps lists, that is redundant, just assume they are there.

In [56]:
reload(h)

def moving_average(differences, window_size):
    """
    Calculate the moving average of the differences with a given window size.

    Args:
        differences (list): List of timestamp differences.
        window_size (int): The window size for the moving average.

    Returns:
        list: The moving average of the differences.
    """
    if window_size < 1:
        raise ValueError("Window size must be greater than or equal to 1.")

    return [sum(differences[i:i+window_size]) / window_size for i in range(len(differences) - window_size + 1)]

# Step 1: Calculate differences without the 5-second threshold
all_differences = [(empatica_ts, closest_inquisit_ts, abs(empatica_ts - closest_inquisit_ts))
                   for empatica_ts in empatica_timestamps
                   for closest_inquisit_ts in [h.closest_timestamp(empatica_ts, inquisit_timestamps)]]

# Step 2: Calculate moving average of the differences
window_size = 5  # Adjust this value based on the expected stability of the synchronization
moving_averages = moving_average([diff for _, _, diff in all_differences], window_size)

# Step 3: Identify the most stable period and compute the average difference in that period
most_stable_period_index = moving_averages.index(min(moving_averages))
most_stable_differences = all_differences[most_stable_period_index:most_stable_period_index + window_size]

# Step 4: Calculate the optimal shift using the average difference in the most stable period
optimal_shift = sum(diff for _, _, diff in most_stable_differences) / len(most_stable_differences)
print(optimal_shift)

0.7746000289916992


In [58]:
reload(h)
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Calculate differences without the 5-second threshold
all_differences = [(empatica_ts, closest_inquisit_ts, abs(empatica_ts - closest_inquisit_ts))
                   for empatica_ts in empatica_timestamps
                   for closest_inquisit_ts in [h.closest_timestamp(empatica_ts, inquisit_timestamps)]]

# Step 2: Perform linear regression between Empatica and Inquisit timestamps
empatica_ts_array = np.array([ts for ts, _, _ in all_differences]).reshape(-1, 1)
inquisit_ts_array = np.array([ts for _, ts, _ in all_differences])

regression_model = LinearRegression()
regression_model.fit(empatica_ts_array, inquisit_ts_array)

# Step 3: Calculate the optimal shift using the intercept of the regression line
optimal_shift = regression_model.intercept_
print(optimal_shift)

104273.97266435623
